In [ ]:
!pip install optuna rtdl category_encoders ruamel.yaml einops

In [ ]:
from scipy.optimize import fmin
import random
import os
import sys
import pandas as pd
import pickle
import pathlib
import argparse
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from tqdm import tqdm, trange
from typing import Optional, Sequence, Tuple, Union, Any, Dict, List
from copy import deepcopy
import enum
import optuna
import rtdl
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import category_encoders as ce
import ruamel.yaml
import math
from collections import OrderedDict, defaultdict
from sklearn.metrics import roc_auc_score, f1_score, precision_score, accuracy_score, recall_score, roc_auc_score, balanced_accuracy_score, log_loss, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline
from scipy.spatial import distance_matrix
from scipy.linalg import qr
from torch.autograd import Function
from einops import rearrange

In [ ]:
def set_global_seed(seed: int) -> None:
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


set_global_seed(42)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## Preprocessings

In [ ]:
def cos_sin(x: torch.Tensor) -> torch.Tensor:
    return torch.cat([torch.cos(x), torch.sin(x)], -1)

def positional(d, pos):
    return torch.Tensor([np.sin(1/10000 ** (2 * int(i / 2) / d) * pos) if i % 2 == 0 else 
                       np.cos(1/10000 ** (2 * int(i / 2) / d) * pos) for i in range(d)])

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, n_features: int = 5000, dropout: float = 0.1, tf: bool=False):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.tf = tf
        self.d_model = d_model
        position = torch.arange(n_features).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(n_features, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        self.pe = pe

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.tf:
            x = x + self.pe.unsqueeze(0)
        else:
            x = x + self.pe.view(1, -1)
        return self.dropout(x)


class Periodic(nn.Module):
    def __init__(self, n_features: int, n: int, sigma: float, trainable: bool, initialization: str, tf: bool) -> None:
        super().__init__()
        self.tf = tf
        if initialization == 'log-linear':
            coefficients = sigma ** (torch.arange(n) / n)
            coefficients = coefficients[None].repeat(n_features, 1)
        else:
            assert initialization == 'normal'
            coefficients = torch.normal(0.0, sigma, (n_features, n))
        if trainable:
            self.coefficients = nn.Parameter(coefficients) 
        else:
            self.register_buffer('coefficients', coefficients)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.tf:
            return cos_sin(2 * np.pi * self.coefficients[None] * x[..., None]).view(-1, 2 * x.shape[1] * self.coefficients.shape[1])
        else:
            return cos_sin(2 * np.pi * self.coefficients[None] * x[..., None]).view(x.shape[0], x.shape[1], 2 * self.coefficients.shape[1])
    
    
class NLinear(nn.Module):
    def __init__(self, n: int, d_in: int, d_out: int, bias: bool = True) -> None:
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(n, d_in, d_out))
        self.bias = nn.Parameter(torch.Tensor(n, d_out)) if bias else None
        with torch.no_grad():
            for i in range(n):
                layer = nn.Linear(d_in, d_out)
                self.weight[i] = layer.weight.T
                if self.bias is not None:
                    self.bias[i] = layer.bias

    def forward(self, x):
        if x.ndim == 3:
            x = x[..., None] * self.weight[None]
            x = x.sum(-2)
            if self.bias is not None:
                x = x + self.bias[None]
            return x
    

class FeaturesTokenizer(nn.Module):
    def __init__(self, n_features: int, d_embedding: int, tf: bool) -> None:
        super().__init__()
        self.tf = tf
        self.first_layer = rtdl.NumericalFeatureTokenizer(n_features, d_embedding, True, 'uniform')
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.first_layer(x)
        if not self.tf:
            return x.view(x.shape[0], -1)
        else:
            return x
    
class LinearEmbeddings(nn.Module):
    def __init__(self, n_layers, n_features, d_embeddings, tf):
        super().__init__()
        self.n_layers = n_layers
        self.tf = tf
        self.layers = nn.ModuleList()
        for i in range(n_layers):
            if i == 0:
                self.layers.append(rtdl.NumericalFeatureTokenizer(n_features, d_embeddings[i], False, 'uniform'))
            else:
                self.layers.append(NLinear(n_features, d_embeddings[i-1], d_embeddings[i], False))
        self.leaky_relu = nn.LeakyReLU()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for i in range(self.n_layers):
            x = self.layers[i](x)
            x = self.leaky_relu(x)
        if not self.tf:
            return x.view(x.shape[0], -1)
        else:
            return x    
    
    
class AutoDis(nn.Module):
    def __init__(
        self, n_features: int, d_embedding: int, n_meta_embeddings: int, temperature: float, tf: bool
    ) -> None:
        super().__init__()
        self.first_layer = rtdl.NumericalFeatureTokenizer(
            n_features,
            n_meta_embeddings,
            False,
            'uniform',
        )
        self.tf = tf
        self.leaky_relu = nn.LeakyReLU()
        self.second_layer = NLinear(
            n_features, n_meta_embeddings, n_meta_embeddings, False
        )
        self.softmax = nn.Softmax(-1)
        self.temperature = temperature
        self.third_layer = NLinear(
            n_features, n_meta_embeddings, d_embedding, False
        )
        nn.init.uniform_(self.third_layer.weight, 0.01)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.first_layer(x)
        x = self.leaky_relu(x)
        x = self.second_layer(x)
        x = self.softmax(x / self.temperature)
        x = self.third_layer(x)
        if not self.tf:
            return x.view(x.shape[0], -1)
        else:
            return x
    
    
class SoftEmbedding(torch.nn.Module):
    def __init__(self, num_embeddings, embeddings_dim, emb_initializer=None, tf=True):
        super(SoftEmbedding, self).__init__()
        self.embedding_table = torch.nn.Embedding(num_embeddings, embeddings_dim)
        if emb_initializer:
            emb_initializer(self.embedding_table.weight)
        self.projection_layer = torch.nn.Linear(1, num_embeddings, bias=True)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.tf = tf

    def forward(self, input_numeric):
        input_numeric = input_numeric.unsqueeze(-1)
        weights = self.softmax(self.projection_layer(input_numeric))
        soft_one_hot_embeddings = (weights.unsqueeze(-1) * self.embedding_table.weight).sum(-2)
        if not self.tf:
            return soft_one_hot_embeddings.view(soft_one_hot_embeddings.shape[0], -1)
        else:
            return soft_one_hot_embeddings


class EntmaxBisectFunction(Function):
    @classmethod
    def _gp(cls, x, alpha):
        return x ** (alpha - 1)

    @classmethod
    def _gp_inv(cls, y, alpha):
        return y ** (1 / (alpha - 1))

    @classmethod
    def _p(cls, X, alpha):
        return cls._gp_inv(torch.clamp(X, min=0), alpha)

    @classmethod
    def forward(cls, ctx, X, alpha=1.5, dim=-1, n_iter=50, ensure_sum_one=True):

        if not isinstance(alpha, torch.Tensor):
            alpha = torch.tensor(alpha, dtype=X.dtype, device=X.device)

        alpha_shape = list(X.shape)
        alpha_shape[dim] = 1
        alpha = alpha.expand(*alpha_shape)

        ctx.alpha = alpha
        ctx.dim = dim
        d = X.shape[dim]

        X = X * (alpha - 1)

        max_val, _ = X.max(dim=dim, keepdim=True)

        tau_lo = max_val - cls._gp(1, alpha)
        tau_hi = max_val - cls._gp(1 / d, alpha)

        f_lo = cls._p(X - tau_lo, alpha).sum(dim) - 1

        dm = tau_hi - tau_lo

        for it in range(n_iter):

            dm /= 2
            tau_m = tau_lo + dm
            p_m = cls._p(X - tau_m, alpha)
            f_m = p_m.sum(dim) - 1

            mask = (f_m * f_lo >= 0).unsqueeze(dim)
            tau_lo = torch.where(mask, tau_m, tau_lo)

        if ensure_sum_one:
            p_m /= p_m.sum(dim=dim).unsqueeze(dim=dim)

        ctx.save_for_backward(p_m)

        return p_m

    @classmethod
    def backward(cls, ctx, dY):
        Y, = ctx.saved_tensors

        gppr = torch.where(Y > 0, Y ** (2 - ctx.alpha), Y.new_zeros(1))

        dX = dY * gppr
        q = dX.sum(ctx.dim) / gppr.sum(ctx.dim)
        q = q.unsqueeze(ctx.dim)
        dX -= q * gppr

        d_alpha = None
        if ctx.needs_input_grad[1]:

            S = torch.where(Y > 0, Y * torch.log(Y), Y.new_zeros(1))
            ent = S.sum(ctx.dim).unsqueeze(ctx.dim)
            Y_skewed = gppr / gppr.sum(ctx.dim).unsqueeze(ctx.dim)

            d_alpha = dY * (Y - Y_skewed) / ((ctx.alpha - 1) ** 2)
            d_alpha -= dY * (S - Y_skewed * ent) / (ctx.alpha - 1)
            d_alpha = d_alpha.sum(ctx.dim).unsqueeze(ctx.dim)

        return dX, d_alpha, None, None, None

        
        
        
def entmax_bisect(X, alpha=1.5, dim=-1, n_iter=50, ensure_sum_one=True):
    return EntmaxBisectFunction.apply(X, alpha, dim, n_iter, ensure_sum_one)

        
        
class EntmaxBisect(nn.Module):
    def __init__(self, alpha=1.5, dim=-1, n_iter=50):
        self.dim = dim
        self.n_iter = n_iter
        self.alpha = alpha
        super().__init__()

    def forward(self, X):
        return entmax_bisect(
            X, alpha=self.alpha, dim=self.dim, n_iter=self.n_iter
        )
  
        
class SparseAttLayer(nn.Module):
    def __init__(self, nhead: int, nfield: int, nemb: int, d_k: int, nhid: int, alpha: float = 1.5):
        """ Multi-Head Sparse Attention Layer """
        super(SparseAttLayer, self).__init__()
        self.sparsemax = nn.Softmax(dim=-1) if alpha == 1. \
            else EntmaxBisect(alpha, dim=-1)

        self.scale = d_k ** -0.5
        self.bilinear_w = nn.Parameter(torch.zeros(nhead, nemb, d_k))                   
        self.query = nn.Parameter(torch.zeros(nhead, nhid, d_k))                        
        self.values = nn.Parameter(torch.zeros(nhead, nhid, nfield))                    
        self.reset_parameters()

    def reset_parameters(self) -> None:
        nn.init.xavier_uniform_(self.bilinear_w, gain=1.414)
        nn.init.xavier_uniform_(self.query, gain=1.414)
        nn.init.xavier_uniform_(self.values, gain=1.414)

    def forward(self, x):
        keys = x                                                                        
        att_gates = torch.einsum('bfx,kxy,koy->bkof',
                                 keys, self.bilinear_w, self.query) * self.scale        
        sparse_gates = self.sparsemax(att_gates)                                        
        return torch.einsum('bkof,kof->bkof', sparse_gates, self.values)

    
class Embedding(nn.Module):

    def __init__(self, nfeat, nemb):
        super().__init__()
        self.embedding = nn.Embedding(nfeat, nemb)
        nn.init.xavier_uniform_(self.embedding.weight)

    def forward(self, x):
        emb = self.embedding(x['id'])                           
        return emb * x['value'].unsqueeze(2)                    

def exists(val):
    return val is not None

def default(val, d):
    return val if exists(val) else d

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class GEGLU(nn.Module):
    def forward(self, x):
        x, gates = x.chunk(2, dim = -1)
        return x * F.gelu(gates)

class FeedForward(nn.Module):
    def __init__(self, dim, mult = 4, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult * 2),
            GEGLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim)
        )

    def forward(self, x, **kwargs):
        return self.net(x)

class Attention(nn.Module):
    def __init__(
        self,
        dim,
        heads = 8,
        dim_head = 16,
        dropout = 0.
    ):
        super().__init__()
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head ** -0.5

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Linear(inner_dim, dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h = self.heads
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), (q, k, v))
        sim = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        attn = sim.softmax(dim = -1)
        dropped_attn = self.dropout(attn)

        out = einsum('b h i j, b h j d -> b h i d', dropped_attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)', h = h)
        return self.to_out(out), attn


class Transformer(nn.Module):
    def __init__(self, num_tokens, dim, depth, heads, dim_head, attn_dropout, ff_dropout):
        super().__init__()
        self.embeds = nn.Embedding(num_tokens, dim)
        self.layers = nn.ModuleList([])

        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = attn_dropout)),
                PreNorm(dim, FeedForward(dim, dropout = ff_dropout)),
            ]))

    def forward(self, x, return_attn = False):
        x = self.embeds(x)

        post_softmax_attns = []

        for attn, ff in self.layers:
            attn_out, post_softmax_attn = attn(x)
            post_softmax_attns.append(post_softmax_attn)

            x = x + attn_out
            x = ff(x) + x

        if not return_attn:
            return x

        return x, torch.stack(post_softmax_attns)

class MLP_TT(nn.Module):
    def __init__(self, dims, act = None):
        super().__init__()
        dims_pairs = list(zip(dims[:-1], dims[1:]))
        layers = []
        for ind, (dim_in, dim_out) in enumerate(dims_pairs):
            is_last = ind >= (len(dims_pairs) - 1)
            linear = nn.Linear(dim_in, dim_out)
            layers.append(linear)

            if is_last:
                continue

            act = default(act, nn.ReLU())
            layers.append(act)

        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        return self.mlp(x)


class TabTransformer(nn.Module):
    def __init__(
        self,
        *,
        categories,
        num_continuous,
        dim, 
        depth, 
        heads, 
        dim_head = 16,
        dim_out = 1,
        mlp_hidden_mults = (4, 2),
        mlp_act = None,
        num_special_tokens = 2,
        continuous_mean_std = None,
        attn_dropout = 0.,
        ff_dropout = 0.
    ):
        super().__init__()

        self.num_categories = len(categories)
        self.num_unique_categories = sum(categories)

        self.num_special_tokens = num_special_tokens
        total_tokens = self.num_unique_categories + num_special_tokens

        if self.num_unique_categories > 0:
            categories_offset = F.pad(torch.tensor(list(categories)), (1, 0), value = num_special_tokens)
            categories_offset = categories_offset.cumsum(dim = -1)[:-1]
            self.register_buffer('categories_offset', categories_offset)

        self.num_continuous = num_continuous

        if self.num_continuous > 0:
            if exists(continuous_mean_std):
                assert continuous_mean_std.shape == (num_continuous, 2), f'continuous_mean_std must have a shape of ({num_continuous}, 2) where the last dimension contains the mean and variance respectively'
            self.register_buffer('continuous_mean_std', continuous_mean_std)

            self.norm = nn.LayerNorm(num_continuous)


        self.transformer = Transformer(
            num_tokens = total_tokens,
            dim = dim,
            depth = depth,
            heads = heads,
            dim_head = dim_head,
            attn_dropout = attn_dropout,
            ff_dropout = ff_dropout
        )

        input_size = (dim * self.num_categories) + num_continuous
        l = input_size // 8

        hidden_dimensions = list(map(lambda t: l * t, mlp_hidden_mults))
        all_dimensions = [input_size, *hidden_dimensions, dim_out]
        
        
        self.mlp = MLP_TT(all_dimensions, act = mlp_act)

    def forward(self, x_categ, x_cont, return_attn = False):
        xs = []

        assert x_categ.shape[-1] == self.num_categories, f'you must pass in {self.num_categories} values for your categories input'

        if self.num_unique_categories > 0:
            x_categ += self.categories_offset

            x, attns = self.transformer(x_categ, return_attn = True)
            xs.append(x)
        if self.num_continuous > 0:
            if exists(self.continuous_mean_std):
                mean, std = self.continuous_mean_std.unbind(dim = -1)
                x_cont = (x_cont - mean) / std

            normed_cont = self.norm(x_cont)
            xs.append(normed_cont)

        x = torch.cat(xs, dim = -1)
        return x
        


## Models

In [ ]:
class _TokenInitialization(enum.Enum):
    UNIFORM = 'uniform'
    NORMAL = 'normal'

    @classmethod
    def from_str(cls, initialization: str) -> '_TokenInitialization':
        try:
            return cls(initialization)
        except ValueError:
            valid_values = [x.value for x in _TokenInitialization]
            raise ValueError(f'initialization must be one of {valid_values}')

    def apply(self, x: torch.Tensor, d: int) -> None:
        d_sqrt_inv = 1 / math.sqrt(d)
        if self == _TokenInitialization.UNIFORM:
            nn.init.uniform_(x, a=-d_sqrt_inv, b=d_sqrt_inv)
        elif self == _TokenInitialization.NORMAL:
            nn.init.normal_(x, std=d_sqrt_inv)

class CategoricalFeatureTokenizer(nn.Module):
    category_offsets: torch.Tensor

    def __init__(
        self,
        cardinalities: List[int],
        d_token: int,
        bias: bool,
        initialization: str,
    ) -> None:
        super().__init__()
        assert cardinalities, 'cardinalities must be non-empty'
        assert d_token > 0, 'd_token must be positive'
        initialization_ = _TokenInitialization.from_str(initialization)

        category_offsets = torch.tensor([0] + cardinalities[:-1]).cumsum(0)
        self.register_buffer('category_offsets', category_offsets, persistent=False)
        self.embeddings = nn.Embedding(sum(cardinalities), d_token)
        self.bias = nn.Parameter(torch.Tensor(len(cardinalities), d_token)) if bias else None

        for parameter in [self.embeddings.weight, self.bias]:
            if parameter is not None:
                initialization_.apply(parameter, d_token)

    @property
    def n_tokens(self) -> int:
        return len(self.category_offsets)

    @property
    def d_token(self) -> int:
        return self.embeddings.embedding_dim

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embeddings(x + self.category_offsets[None])
        if self.bias is not None:
            x = x + self.bias[None]
        return x


class FeatureTokenizer(nn.Module):
    def __init__(
        self,
        n_num_features: int,
        cat_cardinalities: List[int],
        d_token: int,
        preproc_type: str,
        cat_preproc_type: str,
        preproc_args: dict,
        positional: bool,
        tf: bool
    ) -> None:
        super().__init__()
        assert n_num_features >= 0, 'n_num_features must be non-negative'
        assert (
            n_num_features or cat_cardinalities
        ), 'at least one of n_num_features or cat_cardinalities must be positive/non-empty'
        self.initialization = 'uniform'
        self.n_num_features = n_num_features
        self.d = d_token
        self.tf = tf
        self.args = preproc_args
        self.preproc_type = preproc_type
        self.cat_preproc_type = cat_preproc_type
        self.arm_num = None
        self.arm_num_cat = None
        if preproc_type == 'ARM' or preproc_type == 'ARM_Bin':
            self.attn_layer = SparseAttLayer(8, n_num_features, d_token, d_token, 16, 1.7)
            self.arm_num = 8 * self.d * 16 
        if n_num_features:
            if preproc_type == 'Periodic':
                self.num_tokenizer = Periodic(**preproc_args)
                self.d = self.d * 2
            elif preproc_type == 'Linear':
                self.num_tokenizer = LinearEmbeddings(**preproc_args)
            elif preproc_type == 'AutoDis':
                self.num_tokenizer = AutoDis(**preproc_args)
            elif preproc_type == 'Tokens' or preproc_type == 'ARM':
                self.num_tokenizer = FeaturesTokenizer(**preproc_args)
            elif preproc_type == 'SoftEmbedding':
                self.num_tokenizer = SoftEmbedding(**preproc_args)
            elif preproc_type == 'BinEncoding' or preproc_type == 'ARM_Bin':
                self.num_tokenizer = BinEncoding(**preproc_args)
            elif preproc_type == 'None':
                self.num_tokenizer = None
        else:
            self.num_tokenizer = None
        if cat_cardinalities is None:
            self.cat_tokenizer = None
        elif cat_preproc_type == 'Lookup' or cat_preproc_type == 'ARM': 
            self.cat_tokenizer = (
                CategoricalFeatureTokenizer(
                    cat_cardinalities, self.d, True, self.initialization
                )
                if cat_cardinalities
                else None
            )
        elif cat_preproc_type == 'TT':
            self.cat_tokenizer = (
                TabTransformer(
                    num_continuous=0,
                    categories=cat_cardinalities, 
                    dim=self.d, 
                    depth=3,
                    heads=8,
                )
                if cat_cardinalities
                else None
            )
        if cat_preproc_type == 'ARM':
            self.arm_num_cat = (self.n_num_features + 8 * 16) * self.d
            self.cat_attn_layer = SparseAttLayer(8, len(cat_cardinalities), self.d, self.d, 16, 1.7)
        if positional:
            self.positional = PositionalEncoding(self.d, self.n_num_features, 0.1, True)
        else:
            self.positional = None

    @property
    def n_tokens(self) -> int:
        return self.n_num_features + self.cat_tokenizer.n_tokens
        
    @property
    def d_token(self) -> int:
        return self.d

    def forward(self, x_num: Optional[torch.Tensor], x_cat: Optional[torch.Tensor]) -> torch.Tensor:
        if self.num_tokenizer is None:
            return x_num[..., None]
        assert (
            x_num is not None or x_cat is not None
        ), 'At least one of x_num and x_cat must be presented'
        assert _all_or_none(
            [self.num_tokenizer, x_num]
        ), 'If self.num_tokenizer is (not) None, then x_num must (not) be None'
        assert _all_or_none(
            [self.cat_tokenizer, x_cat]
        ), 'If self.cat_tokenizer is (not) None, then x_cat must (not) be None'
        x = []
        if self.num_tokenizer is not None:
            if self.positional is not None:
                x.append(self.positional(self.num_tokenizer(x_num)))
            else:
                x.append(self.num_tokenizer(x_num))
        if self.cat_tokenizer is not None:
            if self.cat_preproc_type == 'TT':
                x.append(self.cat_tokenizer(x_cat, None))
            elif self.cat_preproc_type == 'ARM':
                temp = self.cat_tokenizer(x_cat)
                w = self.cat_attn_layer(temp)
                w = torch.clamp(w, -1e5, 1e5)
                x_cat_arm = torch.exp(torch.einsum('bfe,bkof->bkoe', temp, w))          
                x_cat_arm = rearrange(x_cat_arm, 'b k o e -> b (k o) e')
                x_cat_arm = torch.clamp(x_cat_arm, -1e5, 1e5)
                x.append(x_cat_arm)
            else:
                x.append(self.cat_tokenizer(x_cat))

        if self.preproc_type == 'ARM' or self.preproc_type == 'ARM_Bin':
            e = x[0]
            arm_weight = self.attn_layer(e)
            arm_weight = torch.clamp(arm_weight, -1e5, 1e5)                             
            x_arm = torch.exp(torch.einsum('bfe,bkof->bkoe', e, arm_weight))         
            x_arm = rearrange(x_arm, 'b k o e -> b (k o) e')
            x_arm = torch.clamp(x_arm, -1e5, 1e5)
            x[0] = x_arm
            x = torch.cat(x, dim=1)
            if not self.tf:
                return x.reshape(x.shape[0], -1)
            else:
                return x
        if not self.tf:
            return x[0].view(x[0].shape[0], -1) if len(x) == 1 else torch.cat(x, dim=1).view(x[0].shape[0], -1)
        else:
            return x[0] if len(x) == 1 else torch.cat(x, dim=1)


In [ ]:
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union, cast

def reglu(x: torch.Tensor) -> torch.Tensor:
    assert x.shape[-1] % 2 == 0
    a, b = x.chunk(2, dim=-1)
    return a * F.relu(b)


def geglu(x: torch.Tensor) -> torch.Tensor:
    assert x.shape[-1] % 2 == 0
    a, b = x.chunk(2, dim=-1)
    return a * F.gelu(b)



ModuleType = Union[str, Callable[..., nn.Module]]
_INTERNAL_ERROR_MESSAGE = 'Internal error. Please, open an issue.'


def _is_glu_activation(activation: ModuleType):
    return (
        isinstance(activation, str)
        and activation.endswith('GLU')
        or activation in [ReGLU, GEGLU]
    )


def _all_or_none(values):
    return all(x is None for x in values) or all(x is not None for x in values)


class ReGLU(nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return reglu(x)


class GEGLU(nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return geglu(x)


In [ ]:
class GaussianNoise(nn.Module):
    def __init__(self, stddev, device):
        super().__init__()
        self.stddev = stddev
        self.device = device

    def forward(self, din):
        if self.training:
            return din + torch.autograd.Variable(
                torch.randn(din.size()).to(self.device) * self.stddev
            )
        return din

class DenseLightBlock(nn.Module):
    def __init__(
        self,
        n_in,
        n_out,
        drop_rate=0.1,
        noise_std=0.05,
        act_fun=nn.ReLU,
        use_bn=True,
        use_noise=True,
        use_dropout=True,
        use_act=True,
        device=torch.device("cuda:0"),
        **kwargs,
    ):
        super(DenseLightBlock, self).__init__()
        self.features = nn.Sequential(OrderedDict([]))

        if use_bn:
            self.features.add_module("norm", nn.BatchNorm1d(n_in))
        if use_dropout and drop_rate:
            self.features.add_module("dropout", nn.Dropout(p=drop_rate))
        if use_noise:
            self.features.add_module("noise", GaussianNoise(noise_std, device))

        self.features.add_module("dense", nn.Linear(n_in, n_out))

        if use_act:
            self.features.add_module("act", act_fun())

    def forward(self, x):
        for name, layer in self.features.named_children():
            x = layer(x)
        return x


class DenseLightModel(nn.Module):
    def __init__(
        self,
        n_in,
        n_out=1,
        hidden_size=[
            512,
            512,
            512,
        ],
        drop_rate=[
            0.1,
            0.1,
            0.1,
        ],
        act_fun=nn.ReLU,
        noise_std=0.05,
        bias=None,
        num_init_features=None,
        use_bn=True,
        use_noise=True,
        use_dropout=True,
        use_act=True,
        concat_input=True,
        device=torch.device("cuda:0"),
        preproc=False,
        preproc_type=None,
        preproc_args=None,
        positional=False,
        tokenizer=None,
        **kwargs,
    ):
        super(DenseLightModel, self).__init__()
        
        if isinstance(drop_rate, float):
            drop_rate = [drop_rate] * len(hidden_size)
            
        assert len(hidden_size) == len(
            drop_rate
        ), "Wrong number hidden_sizes/drop_rates. Must be equal."

        self.concat_input = concat_input
        num_features = n_in if num_init_features is None else num_init_features

        self.features = nn.Sequential(OrderedDict([]))
        self.d_embedding = None
        self.tokenizer = tokenizer
        if tokenizer is not None:
            self.features.add_module('preproc', tokenizer)
            if tokenizer.arm_num is not None:
                num_features = tokenizer.arm_num
            elif tokenizer.arm_num_cat is not None:
                num_features = tokenizer.arm_num_cat
            else:
                num_features = num_features * tokenizer.d
            self.d_embedding = tokenizer.d
        if preproc:
            if preproc_type == 'Periodic':
                self.features.add_module('preproc', Periodic(**preproc_args))
                num_features = 2 * num_features * preproc_args['n']
                self.d_embedding = preproc_args['n']
            elif preproc_type == 'Linear':
                self.features.add_module('preproc', LinearEmbeddings(**preproc_args))
                num_features = num_features * preproc_args['d_embeddings'][-1]
                self.d_embedding = preproc_args['d_embeddings'][-1]
            elif preproc_type == 'AutoDis':
                self.features.add_module('preproc', AutoDis(**preproc_args))
                num_features = num_features * preproc_args['d_embedding']
                self.d_embedding = preproc_args['d_embedding']
            elif preproc_type == 'Tokens':
                self.features.add_module('preproc', FeaturesTokenizer(**preproc_args))
                num_features = num_features * preproc_args['d_embedding']
                self.d_embedding = preproc_args['d_embedding']
            elif preproc_type == 'SoftEmbedding':
                self.features.add_module('preproc', SoftEmbedding(**preproc_args))
                num_features = num_features * preproc_args['embeddings_dim']
                self.d_embedding = preproc_args['embeddings_dim']
        if positional:
            self.features.add_module('positional', PositionalEncoding(self.d_embedding, n_in, 0.1, False))
        if num_init_features is not None:
            self.features.add_module("dense0", nn.Linear(n_in, num_features))

        for i, hid_size in enumerate(hidden_size):
            block = DenseLightBlock(
                n_in=num_features,
                n_out=hid_size,
                drop_rate=drop_rate[i] if use_dropout else 0,
                noise_std=noise_std,
                act_fun=act_fun,
                use_bn=use_bn,
                use_noise=use_noise,
                use_dropout=use_dropout,
                use_act=use_act,
                device=device,
            )
            self.features.add_module("denseblock%d" % (i + 1), block)

            if concat_input:
                num_features = n_in + hid_size
            else:
                num_features = hid_size

        num_features = hidden_size[-1]
        self.fc = nn.Linear(num_features, n_out)

        if bias is not None:
            print("init bias!")
            bias = torch.Tensor(bias)
            self.fc.bias.data = bias
            self.fc.weight.data = torch.zeros(n_out, num_features, requires_grad=True)

    def forward(self, x, x_cat=None):
        for name, layer in self.features.named_children():
            if name == 'preproc' and self.tokenizer is not None:
                x = layer(x, x_cat)
            else:
                x = layer(x)
        x = self.fc(x)
        return x


class MLP(DenseLightModel):
    def __init__(self, *args, **kwargs):
        super(MLP, self).__init__(*args, **{**kwargs, **{"concat_input": False}})

    def forward(self, x, x_cat=None):
        return super(MLP, self).forward(x, x_cat)


## Metrics

In [ ]:
class Metric():
    def __init__(self, metric, higher_is_better=True, name='name', optimize=False, discrete=False, **kwargs):
        self.name = name
        self.higher_is_better = higher_is_better
        self.optimize = optimize
        self.discrete = discrete
        self.metric = metric
        self.best_thr = 0.5

    def __repr__(self):
        return str(self.name)

    def __call__(self, y_true, y_pred, thr=0.5, use_best=False):
        if self.discrete:
            return self.metric(y_true, y_pred, thr=thr if not use_best else self.best_thr)
        else:
            return self.metric(y_true, y_pred)

    def find_threshold(self, y_true, y_pred):
        if self.optimize:
            w0 = [0.5]
            res = fmin(self.opt, w0, args=(y_true, y_pred), disp=0)[0]
            self.best_thr = res
            return res
        else:
            return 0.5

    def opt(self, w, y_true, y_pred):
        return (-1) ** (self.higher_is_better) * self(y_true, y_pred, w[0])


def f1_custom(y_true, y_pred, thr=0.5):
    return f1_score(y_true, y_pred > thr, average='micro')


def f1_macro(y_true, y_pred, thr=0.5):
    return f1_score(y_true, y_pred > thr, average='macro')


def acc_score(y_true, y_pred, thr=0.5):
    return accuracy_score(y_true, y_pred > thr)


def bacc_score(y_true, y_pred, thr=0.5):
    return balanced_accuracy_score(y_true, y_pred > thr)


class MetricFactory:
    def __init__(self, ):
        self.metrics = {
            'auc': Metric(metric=roc_auc_score, higher_is_better=True, name='auc', optimize=False, discrete=False),
            'log-loss': Metric(metric=log_loss, higher_is_better=False, name='log-loss', optimize=False,
                              discrete=False),
            'f1': Metric(metric=f1_custom, higher_is_better=True, name='f1', optimize=True, discrete=True),
            'f1-macro': Metric(metric=f1_macro, higher_is_better=True, name='f1_macro', optimize=True, discrete=True),
            'balanced-acc': Metric(metric=bacc_score, higher_is_better=True, name='balanced-acc', optimize=True,
                                   discrete=True),
            'acc': Metric(metric=acc_score, higher_is_better=True, name='acc', optimize=True, discrete=True),
            'mse': Metric(metric=mean_squared_error, higher_is_better=False, name='mse', optimize=False, discrete=False),
            'r2': Metric(metric=r2_score, higher_is_better=True, name='r2', optimize=False, discrete=False),
            'mae': Metric(metric=mean_absolute_error, higher_is_better=False, name='mae', optimize=False, discrete=False)
        }

    def get_allowed(self):
        return sorted(list(self.metrics.keys()))

    def add(self, metric_name, metric_class):
        self.metrics[metric_name] = metric_class
        return self

    def remove(self, metric_name):
        del self.models[metric_name]
        return self

    def __getitem__(self, metric_name):
        return deepcopy(self.metrics[metric_name])


## Train and evaluation

In [ ]:
def compute_metrics(
    outputs,
    targets,
) -> Dict[str, float]:
    metrics = {}
    
    y_true = np.array(targets.cpu())
    y_pred = (np.array(outputs.detach().cpu()))
    
    auc = metric_factory['auc']
    auc_score = auc(y_true, y_pred)
    metrics['auc'] = auc_score
    return metrics

In [ ]:
def train_epoch_ftt(
    model: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    device: torch.device,
    epoch: int,
    silent: bool
) -> None:
    model.train()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)
    if not silent:
        for i, (data, targets) in tqdm(
            enumerate(dataloader),
            total=len(dataloader),
        ):

            data, targets = data.to(device), targets.to(device)
            data_num = data
            data_cat = None
            optimizer.zero_grad()
            pred = model(data_num, data_cat)
            loss = criterion(pred, targets)
            loss.backward()
            optimizer.step()
    else:
        for i, (data, targets) in enumerate(dataloader):

            data, targets = data.to(device), targets.to(device)
            data_num = data
            data_cat = None
            optimizer.zero_grad()
            pred = model(data_num, data_cat)
            loss = criterion(pred, targets)
            loss.backward()
            optimizer.step()

In [ ]:
def evaluate_epoch_ftt(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    scheduler: torch.optim.lr_scheduler,
    writer: list,
    device: torch.device,
    epoch: int,
    dataset: str,
    glob_silent: bool,
    silent: bool
) -> None:

    model.eval()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)
    
    true_val = None
    pred_val = None

    with torch.no_grad():
        if dataset == 'train':
            desc = 'loop over train batches'
        else:
            desc = 'loop over test batches'

        if not glob_silent:
            for i, (data, targets) in tqdm(
                enumerate(dataloader),
                total=len(dataloader),
                desc=desc,
            ):

                data, targets = data.to(device), targets.to(device)
                data_num = data
                data_cat = None
                outputs = model(data_num, data_cat)
                loss = criterion(outputs, targets)

                epoch_loss.append(loss.item())

                if true_val is None:
                    true_val = targets
                else:
                    true_val = torch.cat((true_val, targets), 0)
                
                if pred_val is None:
                    pred_val = outputs
                else:
                    pred_val = torch.cat((pred_val, outputs), 0)
        else: 
            for i, (data, targets) in enumerate(dataloader):

                data, targets = data.to(device), targets.to(device)
                data_num = data
                data_cat = None
                outputs = model(data_num, data_cat)
                loss = criterion(outputs, targets)

                epoch_loss.append(loss.item())

                if true_val is None:
                    true_val = targets
                else:
                    true_val = torch.cat((true_val, targets), 0)
                
                if pred_val is None:
                    pred_val = outputs
                else:
                    pred_val = torch.cat((pred_val, outputs), 0)
                    
        batch_metrics = compute_metrics(
            outputs = pred_val,
            targets = true_val
        )
        
        for metric_name, metric_value in batch_metrics.items():
            batch_metrics_list[metric_name].append(metric_value)


        loss = criterion(pred_val, true_val)
        if dataset == 'test' and scheduler is not None:
            scheduler.step(loss)
        val_loss = loss.item()
        
        if not silent:
            if dataset == 'train':
                print(f'Train loss: {val_loss}\n')
            else:
                print(f'Test loss: {val_loss}\n')

        writer.append((batch_metrics_list, val_loss))
        
        for metric_name, metric_value_list in batch_metrics_list.items():
            metric_value = metric_value_list[0]
            if not silent:
                if dataset == 'train':
                    print(f'Train {metric_name}: {metric_value}\n')
                else:
                    print(f'Test {metric_name}: {metric_value}\n')

In [ ]:
def train_ftt(
    n_epochs: int,
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    val_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler,
    criterion: torch.nn.Module,
    writer_train: list,
    writer_val: list,
    writer_test: list,
    device: torch.device,
    patience=10,
    silent=False,
    glob_silent=False
) -> None:
    best_epoch = -1
    best_metric = np.NINF
    
    
    for epoch in range(n_epochs):

        if not silent:
            print(f"Epoch [{epoch+1} / {n_epochs}]\n")

        train_epoch_ftt(
            model=model,
            dataloader=train_dataloader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            epoch=epoch,
            silent=silent
        )
        evaluate_epoch_ftt(
            model=model,
            dataloader=train_dataloader,
            criterion=criterion,
            scheduler=scheduler,
            writer=writer_train,
            device=device,
            epoch=epoch,
            dataset='train',
            glob_silent=glob_silent,
            silent=silent
        )
        evaluate_epoch_ftt(
            model=model,
            dataloader=val_dataloader,
            criterion=criterion,
            scheduler=scheduler,
            writer=writer_val,
            device=device,
            epoch=epoch,
            dataset='test',
            glob_silent=glob_silent,
            silent=silent
        )
        temp_metric = writer_val[-1][0]['auc'][0]
        if temp_metric > best_metric:
            best_epoch = epoch
            best_metric = temp_metric
        elif epoch - best_epoch > patience:
            break
    evaluate_epoch_ftt(
            model=model,
            dataloader=test_dataloader,
            criterion=criterion,
            scheduler=scheduler,
            writer=writer_test,
            device=device,
            epoch=None,
            dataset='test',
            glob_silent=glob_silent,
            silent=silent
    )
    auc = writer_test[-1][0]['auc'][0]
    return auc
    

## Data

### Give Me Some Credit

In [ ]:
df_gsc = pd.read_csv('./gsc-training.csv', )
df_gsc = df_gsc.drop(['Unnamed: 0'], axis=1)
df_gsc.fillna(df_gsc.median(), inplace=True)

scaler = StandardScaler()
nf = ['RevolvingUtilizationOfUnsecuredLines', 'DebtRatio', 'MonthlyIncome', 'age']
cf = ['NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']
df_gsc[nf] = scaler.fit_transform(df_gsc[nf])
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

transf = ohe.fit_transform(df_gsc[cf])
tdf = pd.DataFrame(transf, columns=ohe.get_feature_names())
tdf.index = df_gsc.index
df_gsc = df_gsc.drop(cf, axis=1)
df_gsc = pd.concat([df_gsc, tdf], axis=1)
df_gsc_target_name = 'SeriousDlqin2yrs'
df_gsc.shape

### Churn Modelling

In [ ]:
df_cm = pd.read_csv('./Churn_Modelling.csv')
df_cm = df_cm.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df_cm.fillna(df_cm.median(), inplace=True)

scaler = StandardScaler()
nf = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']
cf = ['Geography', 'Gender', 'Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember']
df_cm[nf] = scaler.fit_transform(df_cm[nf])
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

transf = ohe.fit_transform(df_cm[cf])
tdf = pd.DataFrame(transf, columns=ohe.get_feature_names())
tdf.index = df_cm.index
df_cm = df_cm.drop(cf, axis=1)
df_cm = pd.concat([df_cm, tdf], axis=1)

df_cm_target_name = 'Exited'

df_cm.shape

### Vehicle Loan Default

In [ ]:
df_vld = pd.read_csv('./vehicle_loan_default_train.csv')
df_vld = df_vld.drop(['UNIQUEID', 'EMPLOYEE_CODE_ID', 'MOBILENO_AVL_FLAG'], axis=1)
df_vld.fillna(df_vld.median(), inplace=True)

le = LabelEncoder()
df_vld['AVERAGE_ACCT_AGE'] = le.fit_transform(df_vld['AVERAGE_ACCT_AGE'])
df_vld['CREDIT_HISTORY_LENGTH'] = le.fit_transform(df_vld['CREDIT_HISTORY_LENGTH'])


df_vld['DATE_OF_BIRTH'] = pd.to_datetime(df_vld['DATE_OF_BIRTH'], format='%d-%m-%Y')
df_vld['DATE_OF_BIRTH_d'] = df_vld['DATE_OF_BIRTH'].dt.day
df_vld['DATE_OF_BIRTH_m'] = df_vld['DATE_OF_BIRTH'].dt.month
df_vld['DATE_OF_BIRTH_y'] = df_vld['DATE_OF_BIRTH'].dt.year
df_vld = df_vld.drop(['DATE_OF_BIRTH'], axis=1)
df_vld['DISBURSAL_DATE'] = pd.to_datetime(df_vld['DISBURSAL_DATE'], format='%d-%m-%Y')
df_vld['DISBURSAL_DATE_d'] = df_vld['DISBURSAL_DATE'].dt.day
df_vld['DISBURSAL_DATE_m'] = df_vld['DISBURSAL_DATE'].dt.month
df_vld['DISBURSAL_DATE_y'] = df_vld['DISBURSAL_DATE'].dt.year
df_vld = df_vld.drop(['DISBURSAL_DATE'], axis=1)


nf = ['DISBURSED_AMOUNT', 'ASSET_COST', 'LTV', 'BRANCH_ID', 'SUPPLIER_ID', 'MANUFACTURER_ID', 'CURRENT_PINCODE_ID', 
     'PERFORM_CNS_SCORE', 'PRI_CURRENT_BALANCE', 'PRI_SANCTIONED_AMOUNT', 'PRI_DISBURSED_AMOUNT', 'SEC_CURRENT_BALANCE',
     'SEC_SANCTIONED_AMOUNT', 'SEC_DISBURSED_AMOUNT', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 'NO_OF_INQUIRIES', 
      'PRI_NO_OF_ACCTS', 'PRI_ACTIVE_ACCTS', 'PRI_OVERDUE_ACCTS', 'SEC_NO_OF_ACCTS', 'SEC_ACTIVE_ACCTS', 
      'SEC_OVERDUE_ACCTS', 'NEW_ACCTS_IN_LAST_SIX_MONTHS', 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 
     'AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH', 'DATE_OF_BIRTH_d', 'DATE_OF_BIRTH_m', 'DATE_OF_BIRTH_y', 
     'DISBURSAL_DATE_d', 'DISBURSAL_DATE_m', 'DISBURSAL_DATE_y'] 
cf = ['EMPLOYMENT_TYPE', 'STATE_ID', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG', 'DRIVING_FLAG',
     'PASSPORT_FLAG', 'PERFORM_CNS_SCORE_DESCRIPTION']
scaler = StandardScaler()
df_vld[nf] = scaler.fit_transform(df_vld[nf])
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

transf = ohe.fit_transform(df_vld[cf])
tdf = pd.DataFrame(transf, columns=ohe.get_feature_names())
tdf.index = df_vld.index
df_vld = df_vld.drop(cf, axis=1)
df_vld = pd.concat([df_vld, tdf], axis=1)

df_vld_target_name = 'LOAN_DEFAULT'

df_vld.shape

### Adult Income Dataset

In [ ]:
df_ai = pd.read_csv('./adult.csv')
df_ai.replace('<=50K', 0, inplace=True)
df_ai.replace('>50K', 1, inplace=True)
df_ai.fillna(df_ai.median(), inplace=True)
nf = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week'] 
cf = ['workclass', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender',
     'native-country']
scaler = StandardScaler()
df_ai[nf] = scaler.fit_transform(df_ai[nf])
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

transf = ohe.fit_transform(df_ai[cf])
tdf = pd.DataFrame(transf, columns=ohe.get_feature_names())
tdf.index = df_ai.index
df_ai = df_ai.drop(cf, axis=1)
df_ai = pd.concat([df_ai, tdf], axis=1)

df_ai_target_name = 'income'
df_ai.shape

### HELOC

In [ ]:
df_heloc = pd.read_csv('./RiskData.csv')
df_heloc.replace('.', 0, inplace=True)
df_heloc = df_heloc.drop(['Sampling_Weight'], axis=1)

nf = ['Interest_Revenue', 'Application_Date', 'Age', 'Income', 'Debt_Ratio', 'Loan_Amt',
     'Loan_Amt_Req', 'LTV', 'FICO_Score', 'Prior_Custom_Score', 'Current_Custom_Score', 'CB_Age_Oldest_TL',
     'CB_Age_Newest_TL', 'CB_Avg_Mos_File', 'CB_Nb_Sat_TL', 'CB_Pct_Sat_TL', 'CB_Mos_Since_Dlq', 'CB_Max_Dlq_12_Mos',
     'CB_Max_Dlq_Ever', 'CB_Nb_Total_TL', 'CB_Nb_TL_Open_12', 'CB_Pct_IL_TL', 'CB_Nb_Inq_6_Mos', 
     'CB_Nb_Inq_6_Mos_excl_7_Days', 'CB_Rev_Util', 'CB_IL_Util', 'CB_Nb_Rev_TL_w_Bal', 'CB_Nb_IL_TL_w_Bal', 
     'CB_Nb_Rev_Tl_75_Pct_Limit', 'CB_Pct_TL_w_Bal']
cf = ['Nb_Borrowers', 'Region', 'Bank_Relationship', 'CB_Nb_60_Plus_TL', 'CB_Nb_90_Plus_TL']
scaler = StandardScaler()
df_heloc.fillna(df_heloc.median(), inplace=True)
df_heloc[nf] = df_heloc[nf].astype(float)
df_heloc[cf] = df_heloc[cf].astype(str)

df_heloc[nf] = scaler.fit_transform(df_heloc[nf])
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

transf = ohe.fit_transform(df_heloc[cf])
tdf = pd.DataFrame(transf, columns=ohe.get_feature_names())
tdf.index = df_heloc.index
df_heloc = df_heloc.drop(cf, axis=1)
df_heloc = pd.concat([df_heloc, tdf], axis=1)

df_heloc_target_name = 'Risk_Flag'

df_heloc.shape

### Fraud Ecomm

In [ ]:
from sklearn.preprocessing import LabelEncoder


df_fe = pd.read_csv('./fraud_ecomm.csv')
ip = pd.read_csv('./IpAddress_to_Country.csv')
df_fe['signup_time'] = pd.to_datetime(df_fe['signup_time'], format='%Y-%m-%d %H:%M:%S')
df_fe['signup_time_y'] = df_fe['signup_time'].dt.year
df_fe['signup_time_mon'] = df_fe['signup_time'].dt.month
df_fe['signup_time_w'] = df_fe['signup_time'].dt.week
df_fe['signup_time_d'] = df_fe['signup_time'].dt.day
df_fe['signup_time_h'] = df_fe['signup_time'].dt.hour
df_fe['signup_time_m'] = df_fe['signup_time'].dt.minute
df_fe['signup_time_s'] = df_fe['signup_time'].dt.second
df_fe['signup_time_wd'] = df_fe['signup_time'].dt.dayofweek
df_fe = df_fe.drop(['signup_time'], axis=1)
df_fe['purchase_time'] = pd.to_datetime(df_fe['purchase_time'], format='%Y-%m-%d %H:%M:%S')
df_fe['purchase_time_y'] = df_fe['purchase_time'].dt.year
df_fe['purchase_time_mon'] = df_fe['purchase_time'].dt.month
df_fe['purchase_time_w'] = df_fe['purchase_time'].dt.week
df_fe['purchase_time_d'] = df_fe['purchase_time'].dt.day
df_fe['purchase_time_h'] = df_fe['purchase_time'].dt.hour
df_fe['purchase_time_m'] = df_fe['purchase_time'].dt.minute
df_fe['purchase_time_s'] = df_fe['purchase_time'].dt.second
df_fe['purchase_time_wd'] = df_fe['purchase_time'].dt.dayofweek
df_fe = df_fe.drop(['purchase_time'], axis=1)
le = LabelEncoder()
df_fe['device_id'] = le.fit_transform(df_fe['device_id'])
df_fe['source'] = le.fit_transform(df_fe['source'])
df_fe['browser'] = le.fit_transform(df_fe['browser'])
df_fe['sex'] = le.fit_transform(df_fe['sex'])
df_fe['age'] = le.fit_transform(df_fe['age'])
ip['country'] = le.fit_transform(ip['country'])
ip['lower_bound_ip_address'] = ip['lower_bound_ip_address'].astype('float')
ip['upper_bound_ip_address'] = ip['upper_bound_ip_address'].astype('float')
df_fe['ip_address'] = df_fe['ip_address'].astype('float')
def ip_to_country(ip_val):
    try :
        return ip.country[(ip.lower_bound_ip_address < ip_val)                            
                                & 
                                (ip.upper_bound_ip_address > ip_val)].iloc[0]
    except IndexError :
        return -1
df_fe['ip_country'] = df_fe['ip_address'].apply(ip_to_country)

device_duplicates = pd.DataFrame(df_fe.groupby(by = "device_id").device_id.count())
device_duplicates.rename(columns={"device_id": "freq_device"}, inplace=True)           
device_duplicates.reset_index(level=0, inplace= True)
df_fe = df_fe.merge(device_duplicates, on= "device_id")

df_fe = df_fe.drop(['user_id'], axis=1)
nf = ['purchase_value', 'ip_address', 'device_id', 'signup_time_y', 'signup_time_mon',
       'signup_time_w', 'signup_time_d', 'signup_time_h', 'signup_time_m',
       'signup_time_s', 'signup_time_wd', 'purchase_time_y',
       'purchase_time_mon', 'purchase_time_w', 'purchase_time_d',
       'purchase_time_h', 'purchase_time_m', 'purchase_time_s',
       'purchase_time_wd', 'age', 'ip_country']
cf = ['source', 'browser', 'sex']

scaler = StandardScaler()
df_fe[nf] = scaler.fit_transform(df_fe[nf])
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

transf = ohe.fit_transform(df_fe[cf])
tdf = pd.DataFrame(transf, columns=ohe.get_feature_names())
tdf.index = df_fe.index
df_fe = df_fe.drop(cf, axis=1)
df_fe = pd.concat([df_fe, tdf], axis=1)

df_fe_target_name = 'class'

df_fe.shape

## Runs

In [ ]:
dfs = {'gsc': df_gsc, 'cm': df_cm, 'heloc': df_heloc, 'vld': df_vld, 'ai': df_ai, 'fe': df_fe}
dfs_shapes = {'gsc': df_gsc.shape[1]-1, 'cm': df_cm.shape[1]-1, 
              'heloc': df_heloc.shape[1]-1, 'vld': df_vld.shape[1]-1, 'ai': df_ai.shape[1]-1, 'fe': df_fe.shape[1]-1}
dfs_targets = {'gsc': df_gsc_target_name, 'cm': df_cm_target_name, 
               'heloc': df_heloc_target_name, 'vld': df_vld_target_name,
              'ai': df_ai_target_name, 'fe': df_fe_target_name}
dfs_names = ['gsc', 'cm', 'vld', 'ai', 'heloc', 'fe']
preproc_types = ['Periodic', 'Fourier', 'Linear', 'AutoDis', 'Tokens', 'SoftEmbedding']

In [ ]:
print(df_gsc.shape)
print(df_cm.shape)
print(df_vld.shape)
print(df_ai.shape)
print(df_heloc.shape)
print(df_fe.shape)

In [ ]:
def get_preproc_params(dataset, preproc_type, tf, d_embedding):
    preproc_params = {}
    if preproc_type == 'Periodic':
        preproc_params = {'n_features': dfs_shapes[dataset], 'n': d_embedding, 'sigma': 0.1, 'trainable': True,
                         'initialization': 'normal', 'tf': tf}
        preproc_name = 'Periodic'
    elif preproc_type == 'Fourier':
        preproc_params = {'n_features': dfs_shapes[dataset], 'n': d_embedding, 'sigma': 0.1, 'trainable': False,
                         'initialization': 'normal', 'tf': tf}
        preproc_name = 'Periodic'
    elif preproc_type == 'Linear':
        preproc_params = {'n_layers': 2, 'n_features': dfs_shapes[dataset], 
                          'd_embeddings': [d_embedding, d_embedding], 'tf': tf}
        preproc_name = 'Linear'
    elif preproc_type == 'AutoDis':
        preproc_params = {'n_features': dfs_shapes[dataset], 'd_embedding': d_embedding, 'n_meta_embeddings': 20,
                         'temperature': 0.5, 'tf': tf}
        preproc_name = 'AutoDis'
    elif preproc_type == 'Tokens' or preproc_type == 'ARM':
        preproc_params = {'n_features': dfs_shapes[dataset], 'd_embedding': d_embedding, 'tf': tf}
        preproc_name = preproc_type
    elif preproc_type == 'SoftEmbedding':
        preproc_params = {'num_embeddings': dfs_shapes[dataset], 'embeddings_dim': d_embedding, 
                          'emb_initializer': None, 'tf': tf}
        preproc_name = 'SoftEmbedding'
    elif preproc_type == 'BinEncoding' or preproc_type == 'ARM_Bin':
        preproc_params = {'bin_edges': bin_edges, 'bins': bins, 'bin_values': bin_values, 'nbins': nbins,
                         'tf': tf, 'd_token': d_embedding, 'bias': True, 'initialization': 'normal', 
                          'device': device}
        preproc_name = 'BinEncoding'
    return preproc_params, preproc_name

In [ ]:
path = './output/bc/mlp/'
positional = False
tf = False
d_embedding = 10

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
%%time
for df_name in tqdm(['gsc']):
    for preproc_type in tqdm(preproc_types):
        temp_path = df_name + '/' + preproc_type + '/'
        X = dfs[df_name].drop([dfs_targets[df_name]], axis=1)
        y = dfs[df_name][dfs_targets[df_name]]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
        X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)
        temp = []
        final_res = defaultdict(list)
        n_runs=15
        silent=True,
        glob_silent=True
        best_metric = np.NINF
        cat_len = 0
        seeds = np.random.randint(1, 100500, 15)
        for i in range(n_runs):
            if not silent:
                print(f'Run {i+1}:')
                print()
            set_global_seed(seeds[i])
            res_metrics = defaultdict(list)   
            X_train_t = torch.from_numpy(X_train_t.values).float()
            y_train_t = torch.from_numpy(y_train_t.values.ravel()).float().unsqueeze(1)
            X_test_t = torch.from_numpy(X_test_t.values).float()
            y_test_t = torch.from_numpy(y_test_t.values.ravel()).float().unsqueeze(1)
            X_test = torch.from_numpy(X_test.values).float()
            y_test = torch.from_numpy(y_test.values.ravel()).float().unsqueeze(1)
        
            train_ds = torch.utils.data.TensorDataset(X_train_t, y_train_t)
            val_ds = torch.utils.data.TensorDataset(X_test_t, y_test_t)
            test_ds = torch.utils.data.TensorDataset(X_test, y_test)
            train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=256, shuffle=True)
            val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=256, shuffle=True)
            test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=256, shuffle=True)
    
            writer_train = []
            writer_val = []
            writer_test = []
      
        
            metric_factory = MetricFactory()
            preproc_params, preproc_name = get_preproc_params(df_name, preproc_type, tf, d_embedding)
                
                
            preproc_params['tf'] = True
            feature_tokenizer = FeatureTokenizer(X.shape[1], None, d_embedding, 
                                                     preproc_name, 'TT', preproc_params, positional, tf)
            model = MLP(n_in=dfs_shapes[df_name], hidden_size=[512, 256, 128], drop_rate=[0.1, 0.1, 0.1], 
                device=device, use_bn=True, use_noise=True, use_dropout=True,
                preproc=False, preproc_type=preproc_name, preproc_args=preproc_params,
                        positional=positional, tokenizer=feature_tokenizer).to(device)

                
            optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=0)
            
            criterion = torch.nn.BCEWithLogitsLoss()
                
            metric = train_ftt(100, model, train_dataloader, val_dataloader, test_dataloader, optimizer, None, criterion, writer_train,
                           writer_val, writer_test, device, 20, silent, glob_silent)
            test_metrics = writer_test[-1][0]
            test_loss = writer_test[-1][1]
            for key, value in test_metrics.items():
                res_metrics[key].append(value[0])
            res_metrics['loss'].append(test_loss)
            for key, value in res_metrics.items():
                final_res[key].append(np.mean(value))
            s = 'run' + str(i + 1) + '.pickle'
            with open(path + temp_path + s, 'wb') as f:
                pickle.dump(res_metrics, f)
    
        ans = {}
        for key, value in final_res.items():
            ans[key] = float(np.mean(value))

        with open(path + temp_path + 'metrics.yml', 'w') as f:
            ruamel.yaml.round_trip_dump(ans, f)